## Arquitectura de módulos del discriminamómetro

Desde este notebook se expone la arquitectura general del proyecto "Discriminamómetro", esto es con la intención de poder correr algún proceso en específico de manera rápida y sencilla.

# ETL
En el módulo ETL se encuentra: 

-El proceso de descarga recurrente que deberá estar programado para ejecutarse cada 15 minutos mediante un cron.
 
-El proceso de obtención de tweets ya sea por usuario, o hashtag/palabras específicas

In [1]:
import Discrim.etl as etl

obj_etl = etl.Etl('PRUEBA')
print(obj_etl.nbr_TweetsXCorrida)

obj_etl.descarga_recurrente_tweets()

8
Entra a descarga_recurrente_tweets
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_apariencia_20200825_023344766864.txt
str_NombreArchivoEnS3:  00_pruebas/json_apariencia_20200825_023344766864.txt
Enviando el archivo a S3...
Envío completado
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_discapacidad_20200825_023345485418.txt
str_NombreArchivoEnS3:  00_pruebas/json_discapacidad_20200825_023345485418.txt
Enviando el archivo a S3...
Envío completado
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_edad_20200825_023346120519.txt
str_NombreArchivoEnS3:  00_pruebas/json_edad_20200825_023346120519.txt
Enviando el archivo a S3...
Envío completado
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_genero_20200825_023346745604.txt
str_NombreArchivoEnS3:  00_prueba

# Train
El "Discriminamómetro" está constituido por 2 capas, en la primer capa se evalúa mediante un modelo si un tweet discrimina o no. En caso de que un tweet sea discriminatorio, en la segunda capa se evaluarán 7 modelos (uno por cada categoría) para determinar el grado de pertenencia a cada una de ellas. Además de ello, para lograr capturar el contexto de cada palabra, se incluye una fase de generación de embeddings que servirá para entrenar los modelos

## Embeddings

In [3]:
import Discrim.Train.embeddings as emb

obj_emb = emb.Embeddings()

obj_emb.descargar_archivo_fuente()
obj_emb.transformar_dataset()
obj_emb.generar_embeddings()
obj_emb.enviar_embeddings_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


str_ArchivoEnvio:  ./data/word2vec_Embeddings.txt
str_NombreArchivoEnS3:  00_entrenamiento/embeddings/word2vec_Embeddings.txt
Enviando el archivo a S3...
Envío completado


# Capa1

In [5]:
! gzip ./data/word2vec_Embeddings.txt

In [6]:
! python -m spacy init-model es data --vectors-loc ./data/word2vec_Embeddings.txt.gz

✔ Successfully created model
53549it [00:09, 5552.76it/s]word2vec_Embeddings.txt.gz
✔ Loaded vectors from data/word2vec_Embeddings.txt.gz
✔ Sucessfully compiled vocab
53767 entries, 53549 vectors


In [1]:
import Discrim.Train.capa1 as cp1

obj_capa1 = cp1.capa1()
obj_capa1.descargar_tweets_entrenamiento()
obj_capa1.transformar_dataset()
obj_capa1.train()
obj_capa1.crear_pickle()
obj_capa1.mandar_pickle_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 4}
grid_search.best_score_:  0.6472299043463667
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    7.3s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


grid_search.best_params_:  {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 50}
grid_search.best_score_:  0.677574089612351
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   34.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


grid_search.best_params_:  {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 16, 'n_estimators': 50}
grid_search.best_score_:  0.6936801476757846
str_ArchivoEnvio:  modelo_capa1.p
str_NombreArchivoEnS3:  01_modelos/capa1/modelo_capa1.p
Enviando el archivo a S3...
Envío completado


In [2]:
import numpy as np

npEmbeddings = obj_capa1.generar_embeddings(obj_capa1.test)

predicted = obj_capa1.best_model.predict(npEmbeddings)

np.mean(predicted == obj_capa1.test.label)

0.665

str_ArchivoEnvio:  modelo_capa1.p
str_NombreArchivoEnS3:  01_modelos/capa1/modelo_capa1.p
Enviando el archivo a S3...
Envío completado


# Capa2
En la segunda capa, se evaluarán 7 modelos

In [1]:
import Discrim.Train.Capa2.apariencia as apa

obj_apariencia = apa.Apariencia()
obj_apariencia.descargar_tweets_entrenamiento()
obj_apariencia.transformar_dataset()
obj_apariencia.train()
#obj_apariencia.crear_pickle()
#obj_apariencia.mandar_pickle_s3()

In [2]:
import Discrim.Train.Capa2.discapacidad as disc

obj_discapacidad = disc.Discapacidad()
obj_discapacidad.descargar_tweets_entrenamiento()
obj_discapacidad.transformar_dataset()
obj_discapacidad.train()
#obj_discapacidad.crear_pickle()
#obj_discapacidad.mandar_pickle_s3()

In [3]:
import Discrim.Train.Capa2.edad as edad

obj_edad = edad.Edad()
obj_edad.descargar_tweets_entrenamiento()
obj_edad.transformar_dataset()
obj_edad.train()
#obj_edad.crear_pickle()
#obj_edad.mandar_pickle_s3()

In [ ]:
import Discrim.Train.Capa2.genero as gen

obj_genero = gen.Genero()
obj_genero.descargar_tweets_entrenamiento()
obj_genero.transformar_dataset()
obj_genero.train()
#obj_genero.crear_pickle()
#obj_genero.mandar_pickle_s3()

In [4]:
import Discrim.Train.Capa2.ideologia as ideo

obj_ideologia = ideo.Ideologia()
obj_ideologia.descargar_tweets_entrenamiento()
obj_ideologia.transformar_dataset()
obj_ideologia.train()
#obj_ideologia.crear_pickle()
#obj_ideologia.mandar_pickle_s3()

In [5]:
import Discrim.Train.Capa2.orientacion as orien

obj_orientacion = orien.Orientacion()
obj_orientacion.descargar_tweets_entrenamiento()
obj_orientacion.transformar_dataset()
obj_orientacion.train()
#obj_orientacion.crear_pickle()
#obj_orientacion.mandar_pickle_s3()

In [ ]:
import Discrim.Train.Capa2.religion as relig

obj_religion = relig.Religion()
obj_religion.descargar_tweets_entrenamiento()
obj_religion.transformar_dataset()
obj_religion.train()
#obj_religion.crear_pickle()
#obj_religion.mandar_pickle_s3()

# Predict

Se creará una clase concentradora que tendrá instanciados todos los modelos para que únicamente sea necesario crear un objeto que tenga acceso a todos los clasificadores.

In [1]:
import Discrim.Predict.modelos as mod

obj_modelos = mod.Modelos()

/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
# Capa1
print('obj_modelos.discr.modelo')
print(obj_modelos.discr.modelo)

# Capa2
print('obj_modelos.apar.modelo')
print(obj_modelos.apar.modelo)

print('obj_modelos.discap.modelo')
print(obj_modelos.discap.modelo)

print('obj_modelos.edad.modelo')
print(obj_modelos.edad.modelo)

print('obj_modelos.genero.modelo')
print(obj_modelos.genero.modelo)

print('obj_modelos.ideo.modelo')
print(obj_modelos.ideo.modelo)

print('obj_modelos.orien.modelo')
print(obj_modelos.orien.modelo)

print('obj_modelos.relig.modelo')
print(obj_modelos.relig.modelo)

obj_modelos.discr.modelo
GradientBoostingClassifier(max_depth=4, max_features='sqrt', min_samples_leaf=7,
                           min_samples_split=16, n_estimators=50)
obj_modelos.apar.modelo
GradientBoostingClassifier(learning_rate=0.25, max_depth=7, max_features='sqrt',
                           min_samples_leaf=7, min_samples_split=16,
                           n_estimators=150)
obj_modelos.discap.modelo
GradientBoostingClassifier(max_depth=7, max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=4)
obj_modelos.edad.modelo
GradientBoostingClassifier(learning_rate=0.75, max_depth=6, max_features='sqrt',
                           min_samples_leaf=3, min_samples_split=16)
obj_modelos.genero.modelo
GradientBoostingClassifier(max_depth=7, max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=4)
obj_modelos.ideo.modelo
GradientBoostingClassifier(max_depth=7, max_features='sqrt', min_samples_leaf=3,
                    

# API

La clase API está diseñada para efecutar los 4 tipo de predicciones:

- Predicciones de las últimas horas (en construcción)
- Predicciones por un hashtag (lista)
- Predicciones por un usuario (lista)
- Predicciones de un texto capturado (lista)

Todos los métidos devuelven un diccionario que contiene el resultado de todos los modelos

In [1]:
import Discrim.api as api

# El objeto api recibe la cantidad de tweets que se desean extraer de twitter de acuerdo al tipo de búsqueda
obj_api = api.API(nbr_tweets_x_hrs = 2, nbr_tweets_x_ht = 3, nbr_tweets_x_usr = 4)

/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nl

In [2]:
resultado = obj_api.clasificar_x_horas(-48)
resultado

{'discriminacion': [[0.5444648862930812, 0.45553511370691874],
  [0.9187096849133349, 0.08129031508666501]],
 'apariencia': [[0.8437734998571413, 0.15622650014285863],
  [0.9277879686787409, 0.07221203132125913]],
 'discapacidad': [[0.9207046288857942, 0.07929537111420586],
  [0.9500442965859587, 0.04995570341404137]],
 'edad': [[0.1307144215778484, 0.8692855784221516],
  [0.9960905058499538, 0.0039094941500461364]],
 'genero': [[0.9923863316163615, 0.007613668383638492],
  [0.9940541628418847, 0.005945837158115218]],
 'ideologia': [[0.7367168304735721, 0.26328316952642794],
  [0.8803519263951829, 0.11964807360481709]],
 'orientacion': [[0.9822597141023248, 0.017740285897675144],
  [0.9770046095523691, 0.022995390447630864]],
 'religion': [[0.8159222911776753, 0.1840777088223246],
  [0.9568643210585475, 0.043135678941452545]]}

In [3]:
resultado = obj_api.clasificar_x_ht('#AMLO')
resultado

{'discriminacion': [[0.8230788426016198, 0.17692115739838027],
  [0.36746363010552496, 0.632536369894475],
  [0.9102956879619056, 0.08970431203809437]],
 'apariencia': [[0.7949051396815692, 0.20509486031843077],
  [0.9554016953922145, 0.044598304607785424],
  [0.6723764029316641, 0.3276235970683358]],
 'discapacidad': [[0.8641583685206731, 0.13584163147932693],
  [0.9503374334648361, 0.04966256653516392],
  [0.9187353991634108, 0.08126460083658918]],
 'edad': [[0.981312707648005, 0.01868729235199492],
  [0.9963053799704662, 0.0036946200295338238],
  [0.9928073919479843, 0.007192608052015758]],
 'genero': [[0.9941727260717221, 0.005827273928277826],
  [0.9970250894508034, 0.0029749105491965783],
  [0.9874768098382237, 0.012523190161776278]],
 'ideologia': [[0.8689720533428221, 0.1310279466571779],
  [0.8734078401582422, 0.12659215984175778],
  [0.8357304129703806, 0.16426958702961938]],
 'orientacion': [[0.9710372877187464, 0.0289627122812536],
  [0.9745580750186313, 0.02544192498136869

In [9]:
resultado = obj_api.clasificar_x_usuario('@ChumelTorres')
resultado

{'discriminacion': array([[0.36524777, 0.63475223],
        [0.78139488, 0.21860512]]),
 'apariencia': array([[0.89057011, 0.10942989],
        [0.42667337, 0.57332663]]),
 'discapacidad': array([[0.94288938, 0.05711062],
        [0.76988538, 0.23011462]]),
 'edad': array([[0.986909 , 0.013091 ],
        [0.1907501, 0.8092499]]),
 'genero': array([[0.99677552, 0.00322448],
        [0.99487672, 0.00512328]]),
 'ideologia': array([[0.84315266, 0.15684734],
        [0.78263179, 0.21736821]]),
 'orientacion': array([[0.97147292, 0.02852708],
        [0.96743147, 0.03256853]]),
 'religion': array([[0.97786001, 0.02213999],
        [0.99756776, 0.00243224]])}

In [2]:
resultado = obj_api.clasificar_x_texto('Que chingue a su madre el américa')
resultado

{'discriminacion': [[0.27225924445015426, 0.7277407555498457]],
 'apariencia': [[0.3712523336132679, 0.6287476663867321]],
 'discapacidad': [[0.9067630268506349, 0.09323697314936506]],
 'edad': [[0.2602797200558844, 0.7397202799441156]],
 'genero': [[0.9962268110763076, 0.0037731889236923876]],
 'ideologia': [[0.8704969763088051, 0.12950302369119493]],
 'orientacion': [[0.9804331777476608, 0.01956682225233921]],
 'religion': [[0.5762506723688756, 0.42374932763112433]]}

In [10]:
obj_api.list_data

[{'user_id': '120835612',
  'name': 'Chumel Torres',
  'screen_name': 'ChumelTorres',
  'full_text': 'Cristo de bondad lo chillones que estan por el despido de Ackerman de Proceso.\n\nHaganle como uno. Tomenlo con dignidad. https://t.co/85w3wuM40f',
  'location': 'CDMX'},
 {'user_id': '120835612',
  'name': 'Chumel Torres',
  'screen_name': 'ChumelTorres',
  'full_text': '@jgnaredo @proceso @EnriqueKrauze @aguilarcamin Ahora dilo sin llorar.',
  'location': 'CDMX'}]

In [11]:
obj_api.df

,user_id,name,screen_name,full_text,location,embeddings
0,120835612,Chumel Torres,ChumelTorres,cristo bondad chillones estan despido ackerman...,CDMX,"[-0.2492830455303192, 0.16176076233386993, -0...."
1,120835612,Chumel Torres,ChumelTorres,ahora dilo llorar,CDMX,"[-0.6474771499633789, 0.4220397174358368, -0.2..."


In [12]:
obj_api.X_predict

[array([-2.49283046e-01,  1.61760762e-01, -1.01963378e-01, -6.05144724e-02,
        -3.79346864e-04,  2.57731110e-01,  1.43005699e-02, -8.18011463e-02,
         1.63993359e-01,  1.17667988e-01,  4.00251076e-02, -1.39445901e-01,
        -3.49892117e-02,  3.84578072e-02, -9.70307365e-02,  1.39213102e-02,
        -3.54281403e-02,  1.72622934e-01, -1.14829794e-01, -2.00621814e-01,
         7.21444786e-02,  4.13204543e-02,  8.55631456e-02,  1.37675375e-01,
        -6.09088130e-02,  1.72165111e-01, -1.10027157e-01,  1.77993216e-02,
         1.44955784e-01,  4.52657826e-02, -4.86774966e-02,  9.50034484e-02,
        -8.98902565e-02, -7.44117647e-02,  1.85505357e-02, -2.62994319e-01,
        -3.69041413e-02, -5.28958552e-02,  1.15877666e-01, -1.55869678e-01,
         4.96116653e-02, -1.15810767e-01,  1.94760412e-01, -1.68578535e-01,
         1.82956949e-01, -8.35286267e-03, -2.33858169e-04,  1.02224752e-01,
         1.74121350e-01, -1.93008274e-01,  6.88781887e-02, -7.71545386e-03,
         1.2

# Dashboard

In [6]:
import Discrim.Dashboard as dash